# 콘텐츠 기반 필터링

In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install surprise

In [3]:
from surprise import Dataset
data = Dataset.load_builtin('ml-100k', prompt= False)
df = pd.DataFrame(data.raw_ratings, columns = ['user-id', 'movie-id', 'rating', 'timestamp'])
df.head()

,user-id,movie-id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [4]:
df.shape

(100000, 4)

In [5]:
df['user-id'].min(), df['user-id'].max()

('1', '99')

### 1. Adjacent Matrix (인접행렬) 생성
 - 행은 사용자
 - 열은 영화
 - 내용은 평점 

In [6]:
raw_data=np.array(data.raw_ratings, dtype = int)
np.min(raw_data, axis=0)

array([        1,         1,         1, 874724710])

In [7]:
np.max(raw_data, axis=0)

array([      943,      1682,         5, 893286638])

In [8]:
# user-id, movie-id가 0부터 시작하도록 만들어주는 작업
raw_data[:, :2] -= 1
raw_data[:5]

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       [      243,        50,         2, 880606923],
       [      165,       345,         1, 886397596]])

In [9]:
# 행, 열의 개수
nrows = df['user-id'].nunique() # 행의 개수 = 사용자(user)수
ncols = df['movie-id'].nunique() # 열의 개수 = 영화 개수
nrows, ncols

(943, 1682)

#### 1) 본 영화 / 안 본 영화를 1/0으로구분

In [10]:
# 본 영화는 1로, 안 본 영화는 0으로
adj_matrix = np.zeros([nrows, ncols], int) # 인접 행렬을 (사용자수, 영화개수)로 정의하고 값은 모두 정수

for user_id, movie_id, _, _ in raw_data: # 반복문으로 인접 행렬에 값을 채우는 부분
    adj_matrix[user_id, movie_id] = 1 # 본 영화는 1로 채운다

adj_matrix[:5]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [11]:
# 0번 데이터를 '나'로 가정
my_id, my_vec = 0, adj_matrix[0]

In [12]:
# 유사도 = 이진 벡터의 내적

# 나와 10번, 20번 사용자와의 유사도
np.dot(my_vec, adj_matrix[10]), np.dot(my_vec, adj_matrix[20])

(71, 42)

In [13]:
# 누가 '나'랑 유사한지
best_score, best_match_id = 0, 0

for i in range(1, len(adj_matrix)): # 1번 유저부터 끝까지 비교를 한다
    dot = np.dot(my_vec, adj_matrix[i])

    if dot > best_score: # dot가 0보다 크면 
        best_score, best_match_id = dot, i # best_score의 값과 best_match_id를 바꿔 넣어라

best_score, best_match_id

(183, 275)

In [14]:
best_vec = adj_matrix[best_match_id]
my_vec[200:210], best_vec[200:210]

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1]))

In [15]:
# 내가 보지 않은 영화 중에서 best_match_id 사용자가 본 영화 => 추천
recomm_lst = []

for i ,  (my_view, best_view) in enumerate(zip(my_vec, best_vec)): # 내 벡터와 상대 벡터를 비교해서
    
    if my_view == 0 and best_view == 1: # 내가 안 본 영화이고 best_match_id인 사람이 본 영화인 것만
        recomm_lst.append(i) # 리스트에 넣겠다

len(recomm_lst), recomm_lst[:10]

(335, [272, 273, 275, 280, 281, 283, 287, 288, 289, 290])

 - 평점을 주는 경우

In [16]:
adj_matrix = np.zeros([nrows, ncols], int) # 인접 행렬을 (사용자수, 영화개수)로 정의하고 값은 모두 정수

for user_id, movie_id, rating, _ in raw_data: # 반복문으로 인접 행렬에 값을 채우는 부분
    adj_matrix[user_id, movie_id] = rating # 본 영화의 평점 부여

adj_matrix[:5]

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [4, 3, 0, ..., 0, 0, 0]])

 - Case1: 유클리드 거리
  * 짧을수록 유사도가 높은거

In [17]:
# 누가 '나'랑 유사한지
best_score, best_match_id = 100000, 0
my_vec = adj_matrix[0]

for i in range(1, len(adj_matrix)): # 1번 유저부터 끝까지 비교를 한다
    euc = np.sqrt(np.sum(np.square(my_vec - adj_matrix[i]))) # 유클리드 기하학 거리 계산 공식

# 벡터 거리계산을 이용하여 인접도를 위에서 계산함

    if euc < best_score: # euc가 best_score보다 작으면 
        best_score, best_match_id = euc, i # best_score의 값과 best_match_id를 바꿔 넣어라

best_score, best_match_id

(55.06359959174482, 737)

In [19]:
# 내가 안 본 영화 중에서 best_match_id의 유저가 본 최고 평점 영화 => 추천
best_v = adj_matrix[best_match_id]
recomm_lst = []

for i ,  (my_view, best_view) in enumerate(zip(my_vec, best_v)): # 내 벡터와 상대 벡터를 비교해서
    
    if my_view == 0 and best_view >=4: # 내가 안 본 영화이고 best_match_id인 사람이 본 영화 중 평점이 4점 이상인 영화를
        recomm_lst.append(i) # 리스트에 넣겠다

len(recomm_lst), recomm_lst[:10]

(21, [312, 317, 356, 384, 407, 422, 433, 454, 469, 473])

 - Case 2: Cosine 유사도

In [20]:
# 코사인 유사도 함수 정의
def cos_sim(v1, v2):
    v1_norm = np.sqrt(np.sum(np.square(v1)))
    v2_norm = np.sqrt(np.sum(np.square(v2)))
    return np.dot(v1, v2) / (v1_norm * v2_norm)

In [21]:
# 누가 '나'랑 유사한지
best_score, best_match_id = -1, 0
my_vec = adj_matrix[0]

for i in range(1, len(adj_matrix)): # 1번 유저부터 끝까지 비교를 한다
    cos_similarity = cos_sim(my_vec, adj_matrix[i]) # 유클리드 기하학 거리 계산 공식

# 벡터 거리계산을 이용하여 인접도를 위에서 계산함

    if cos_similarity > best_score: # euc가 best_score보다 작으면 
        best_score, best_match_id =cos_similarity, i # best_score의 값과 best_match_id를 바꿔 넣어라

best_score, best_match_id

(0.569065731527988, 915)